In [ ]:
import os
import sys
import time
import threading
import json
from pathlib import Path
from datetime import datetime
import base64

# Import your backend functions
from FINAL_FILE import (
    enhance_prompt_with_meta_learning,
    get_agent_response,
    asset_identifier_agent,
    high_level_planner_agent,
    spatial_analyzer_agent,
    detailed_planner_agent,
    code_generator_agent,
    code_corrector_agent,
    process_asset_list_sketchfab,
    measure_assets_in_blender,
    run_blender_script,
    validate_scene_with_clip,
    extract_scene_metrics_from_blender,
    store_generation_result,
    store_asset_details,
    store_correction_attempt,
    update_asset_performance,
    parse_json_safely,
    extract_content_from_response,
    safe_json_dumps,
    GROQ_API_KEY,
    SKETCHFAB_API_TOKEN,
    DB_CONFIG
)

try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output, Image as IPImage
except ImportError:
    print("Error: ipywidgets not found. Install with: pip install ipywidgets")
    sys.exit(1)

# Configuration
BLENDER_EXECUTABLE_PATH = r"C:\Program Files\Blender Foundation\Blender 4.2\blender.exe"
ENABLE_META_LEARNING = True
ENABLE_DATABASE = True

# Modern Minimal CSS
CSS_STYLES = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=JetBrains+Mono:wght@300;400;500;600;700&display=swap');

    * {
        font-family: 'JetBrains Mono', monospace !important;
    }

    body {
        background: linear-gradient(135deg, #4f46e5 0%, #7c3aed 100%);
        padding: 20px;
        min-height: 100vh;
    }

    .dashboard-container {
        max-width: 1400px;
        margin: 0 auto;
        background: #ffffff;
        border-radius: 22px;
        padding: 45px;
        box-shadow: 0 25px 70px rgba(0, 0, 0, 0.25);
        border: 1px solid #e5e7eb;
    }

    .header {
        text-align: center;
        margin-bottom: 45px;
    }

    .header h1 {
        font-size: 44px;
        font-weight: 700;
        background: linear-gradient(135deg, #4f46e5 0%, #7c3aed 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        letter-spacing: -1px;
        margin: 0;
    }

    .header p {
        color: #6b7280;
        font-size: 16px;
        margin-top: 8px;
    }

    /* Section Container */
    .section-container {
        background: #ffffff;
        padding: 30px;
        border-radius: 15px;
        border: 1px solid #e5e7eb;
        margin-bottom: 30px;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.08);
        transition: box-shadow 0.3s ease;
    }

    .section-container:hover {
        box-shadow: 0 8px 25px rgba(0, 0, 0, 0.12);
    }

    .section-title {
        font-size: 18px;
        font-weight: 700;
        color: #1f2937;
        margin: 0 0 20px 0;
        padding-bottom: 15px;
        border-bottom: 2px solid #4f46e5;
        display: flex;
        align-items: center;
        gap: 8px;
    }

    .input-section {
        background: #f8fafc;
        padding: 30px;
        border-radius: 15px;
        border: 1px solid #e5e7eb;
        margin-bottom: 30px;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.08);
    }

    /* Progress Bar Container */
    .progress-container {
        background: #ffffff;
        padding: 30px;
        border-radius: 15px;
        border: 2px solid #4f46e5;
        margin: 20px 0;
        box-shadow: 0 6px 20px rgba(79, 70, 229, 0.15);
    }

    .progress-container .widget-hbox {
        display: flex;
        gap: 15px;
        align-items: center;
    }

    .progress-label {
        font-weight: 600;
        color: #4f46e5;
        font-size: 14px;
        min-width: 100px;
    }

    /* Progress Bar Styling */
    .widget-hbox .jupyter-widgets {
        flex: 1 !important;
    }

    .widget-hbox .jupyter-widgets-view {
        flex: 1 !important;
    }

    /* Steps */
    .step-indicator {
        display: flex;
        gap: 10px;
        margin: 20px 0;
        flex-wrap: wrap;
    }

    .step {
        flex: 1;
        min-width: 140px;
        text-align: center;
        padding: 16px;
        background: #f3f4f6;
        border-radius: 12px;
        font-size: 13px;
        font-weight: 600;
        color: #9ca3af;
        border: 1px solid #e5e7eb;
        transition: 0.25s;
    }

    .step.active {
        background: linear-gradient(135deg, #4f46e5 0%, #7c3aed 100%);
        color: white;
        transform: scale(1.06);
        box-shadow: 0 6px 18px rgba(79, 70, 229, 0.35);
        animation: pulse 1.5s ease-in-out infinite;
    }

    @keyframes pulse {
        0%, 100% { opacity: 1; }
        50% { opacity: 0.7; }
    }

    .step.completed {
        background: #10b981;
        color: white;
        border-color: #059669;
    }

    /* Status Boxes */
    .status-box {
        padding: 15px;
        background: #eff6ff;
        border-left: 4px solid #3b82f6;
        border-radius: 10px;
        margin: 12px 0;
        font-size: 14px;
        animation: slideIn 0.3s ease-out;
        box-shadow: 0 2px 8px rgba(59, 130, 246, 0.1);
    }

    .status-box.success {
        background: #f0fdf4;
        border-left-color: #10b981;
        box-shadow: 0 2px 8px rgba(16, 185, 129, 0.1);
    }

    .status-box.warning {
        background: #fffbeb;
        border-left-color: #f59e0b;
        box-shadow: 0 2px 8px rgba(245, 158, 11, 0.1);
    }

    .status-box.error {
        background: #fef2f2;
        border-left-color: #ef4444;
        box-shadow: 0 2px 8px rgba(239, 68, 68, 0.1);
    }

    @keyframes slideIn {
        from {
            opacity: 0;
            transform: translateX(-15px);
        }
        to {
            opacity: 1;
            transform: translateX(0);
        }
    }

    /* Asset Cards */
    .asset-card {
        background: white;
        border: 1px solid #e5e7eb;
        border-radius: 14px;
        padding: 15px;
        margin: 12px 0;
        display: flex;
        gap: 18px;
        align-items: center;
        transition: 0.2s;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.04);
    }

    .asset-card:hover {
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.07);
        transform: translateY(-2px);
    }

    .asset-thumbnail {
        width: 120px;
        height: 120px;
        object-fit: cover;
        border-radius: 10px;
        border: 1px solid #e5e7eb;
    }

    .asset-info {
        flex: 1;
    }

    .asset-name {
        font-size: 16px;
        font-weight: 600;
        color: #1f2937;
        margin-bottom: 4px;
    }

    .asset-score {
        display: inline-block;
        padding: 5px 14px;
        border-radius: 25px;
        font-size: 12px;
        font-weight: 600;
    }

    .score-high {
        background: #d1fae5;
        color: #065f46;
    }

    .score-medium {
        background: #fef3c7;
        color: #92400e;
    }

    .score-low {
        background: #fee2e2;
        color: #991b1b;
    }

    /* Code Section */
    .code-section {
        background: #111827;
        border-radius: 14px;
        padding: 22px;
        margin: 25px 0;
        border: 2px solid #4f46e5;
        box-shadow: 0 6px 20px rgba(79, 70, 229, 0.15);
    }

    .code-section h3 {
        color: white;
        margin-top: 0;
    }

    /* Render Preview */
    .render-preview {
        max-width: 100%;
        border-radius: 14px;
        margin: 20px 0;
        box-shadow: 0 12px 40px rgba(0, 0, 0, 0.25);
    }

    /* Buttons */
    .widget-button button {
        background: linear-gradient(135deg, #4f46e5 0%, #7c3aed 100%) !important;
        color: white !important;
        border: none !important;
        padding: 15px 38px !important;
        font-size: 15px !important;
        font-weight: 600 !important;
        border-radius: 12px !important;
        cursor: pointer !important;
        transition: transform 0.2s, box-shadow 0.2s !important;
        box-shadow: 0 4px 12px rgba(79, 70, 229, 0.3) !important;
    }

    .widget-button button:hover:not(:disabled) {
        transform: translateY(-2px) !important;
        box-shadow: 0 8px 20px rgba(79, 70, 229, 0.4) !important;
    }

    .widget-button button:disabled {
        opacity: 0.6 !important;
        cursor: not-allowed !important;
    }

    /* Textarea */
    .widget-textarea textarea {
        width: 100% !important;
        padding: 15px !important;
        font-size: 14px !important;
        border: 1.5px solid #d1d5db !important;
        border-radius: 12px !important;
        min-height: 110px !important;
        background: #f9fafb !important;
        font-family: 'JetBrains Mono', monospace !important;
        transition: border-color 0.2s !important;
    }

    .widget-textarea textarea:focus {
        border-color: #4f46e5 !important;
        outline: none !important;
        box-shadow: 0 0 0 3px rgba(79, 70, 229, 0.1) !important;
    }

    /* IntProgress - Custom styling */
    .widget-hbox-single .widget-label {
        min-width: auto !important;
        margin-right: 15px !important;
        font-weight: 600 !important;
        color: #4f46e5 !important;
    }

    .widget-hbox-single .widget-inline-hbox {
        flex: 1 !important;
    }

    .progress {
        height: 24px !important;
        border-radius: 12px !important;
        background-color: #f3f4f6 !important;
        border: 1px solid #e5e7eb !important;
        overflow: hidden !important;
        box-shadow: inset 0 2px 4px rgba(0, 0, 0, 0.05) !important;
    }

    .progress-bar {
        background: linear-gradient(90deg, #4f46e5 0%, #7c3aed 100%) !important;
        height: 100% !important;
        border-radius: 12px !important;
        transition: width 0.4s ease !important;
        box-shadow: 0 2px 8px rgba(79, 70, 229, 0.3) !important;
    }

    .progress-bar::after {
        content: attr(data-transitiongoal) !important;
        display: flex !important;
        align-items: center !important;
        justify-content: center !important;
        height: 100% !important;
        color: white !important;
        font-weight: 700 !important;
        font-size: 13px !important;
    }
</style>
"""

class SceneGeneratorDashboard:
    def __init__(self):
        self.generation_running = False
        self.current_generation_id = None
        self.setup_ui()
         
    def setup_ui(self):
        """Initialize all UI components"""
        
        # Header
        self.header = widgets.HTML(
            value='''
            <div style="text-align: center; margin-bottom: 40px;">
                <h1 style="font-size: 42px; font-weight: 700; color: #667eea; margin: 0; padding: 0;">
                    🎨 Meta3DScene - Text-to-3D Scene Generator
                </h1>
                <p style="color: #6b7280; font-size: 16px; margin-top: 10px;">
                    Powered by Meta-Learning & Agentic Workflow
                </p>
            </div>
            '''
        )
        
        # Input Section
        self.prompt_input = widgets.Textarea(
            placeholder='Describe your scene (e.g., "A forest cabin surrounded by pine trees")',
            layout=widgets.Layout(width='100%', height='100px')
        )
        
        self.generate_btn = widgets.Button(
            description='✨ Generate Scene',
            layout=widgets.Layout(width='200px', height='50px')
        )
        self.generate_btn.on_click(self.start_generation)
        
        input_section = widgets.VBox([
            widgets.HTML('<div class="section-title">✍️ Scene Description</div>'),
            self.prompt_input,
            widgets.HBox([self.generate_btn], layout=widgets.Layout(justify_content='center', margin='20px 0'))
        ], layout=widgets.Layout(
            background_color='#f8fafc',
            padding='30px',
            border_radius='15px',
            margin='0 0 30px 0',
            border='1px solid #e5e7eb'
        ))
        
        # Progress Section
        self.steps = [
            "Meta-Learning", "Assets are being Identified", "Assets are being Downloaded & Verifed", "Asset Measurement", 
            "Scene is being Planned", "Blender Script Generating", "Validating the Script", "Scene Validation"
        ]
        self.step_widgets = [
            widgets.HTML(f'<div class="step">{step}</div>') 
            for step in self.steps
        ]
        
        self.step_indicator = widgets.HTML()
        self.update_step_display()
        
        # Progress bar
        self.progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            layout=widgets.Layout(width='100%', height='30px')
        )
        
        # Status output
        self.status_output = widgets.Output()
        
        progress_section = widgets.VBox([
            widgets.HTML('<div class="section-title">📊 Generation Progress</div>'),
            self.step_indicator,
            widgets.HTML('<div style="margin: 20px 0;">'),
            self.progress_bar,
            widgets.HTML('</div>'),
            self.status_output
        ], layout=widgets.Layout(
            background_color='white',
            padding='30px',
            border_radius='15px',
            margin='20px 0',
            border='1px solid #e5e7eb'
        ))
        
        # Asset Display Section
        self.asset_display = widgets.HTML(value='...')
        
        asset_section = widgets.VBox([
            widgets.HTML('<div class="section-title">📦 Downloaded Assets</div>'),
            self.asset_display
        ], layout=widgets.Layout(
            background_color='white',
            padding='30px',
            border_radius='15px',
            margin='20px 0',
            border='1px solid #e5e7eb'
        ))
        
        # Render preview area
        self.render_preview = widgets.HTML(value='<div style="color:#6b7280;">Render previews will appear here...</div>')
        
        render_section = widgets.VBox([
            widgets.HTML('<div class="section-title">🎬 Render Preview</div>'),
            self.render_preview
        ], layout=widgets.Layout(
            background_color='white',
            padding='30px',
            border_radius='15px',
            margin='20px 0',
            border='1px solid #e5e7eb'
        ))
        
        # Code display
        self.code_display = widgets.Textarea(
            value='# Generated Blender code will appear here...',
            disabled=True,
            layout=widgets.Layout(width='100%', height='400px') 
        )
        
        self.copy_btn = widgets.Button(
            description='📋 Copy Code',
            button_style='info',
            layout=widgets.Layout(width='150px', display='none')
        )
        self.copy_btn.on_click(self.copy_code)
        
        code_section = widgets.VBox([
            widgets.HTML('<div class="section-title">📝 Generated Blender Code</div>'),
            self.code_display,
            self.copy_btn
        ], layout=widgets.Layout(
            background_color='#1e1e1e',
            padding='30px',
            border_radius='15px',
            margin='20px 0',
            border='2px solid #4f46e5'
        ))
        
        # Main container with all sections
        self.main_container = widgets.VBox([
            input_section,
            progress_section,
            asset_section,
            render_section,
            code_section
        ])
    
    def _to_files_url(self, path):
        """Return a /files/... URL for path — accepts absolute, relative or basename fallbacks."""
        if not path:
            return ''
        # already a URL
        if path.startswith(("http://", "https://", "/files/")):
            return path
        # absolute path that exists
        if os.path.isabs(path) and os.path.exists(path):
            try:
                rel = os.path.relpath(path, os.getcwd()).replace("\\", "/")
                return "/files/" + rel
            except Exception:
                return ''
        # relative to cwd
        candidate = os.path.join(os.getcwd(), path)
        if os.path.exists(candidate):
            rel = os.path.relpath(candidate, os.getcwd()).replace("\\", "/")
            return "/files/" + rel
        # search workspace for same basename (measurement/ or scene/ etc)
        name = os.path.basename(path)
        for root, _, files in os.walk(os.getcwd()):
            if name in files:
                rel = os.path.relpath(os.path.join(root, name), os.getcwd()).replace("\\", "/")
                return "/files/" + rel
        return ''

    def add_status(self, message, msg_type='info'):
        """Add status message"""
        type_class = {
            'info': 'status-box',
            'success': 'status-box success',
            'warning': 'status-box warning',
            'error': 'status-box error'
        }.get(msg_type, 'status-box')
        
        with self.status_output:
            display(HTML(f'<div class="{type_class}">{message}</div>'))
    
    def update_step(self, idx, status='active'):
        """Update step indicator with animated symbols"""
        if status == 'active':
            spinner_symbol = '⟳'
            self.step_widgets[idx].value = f'<div class="step active">{spinner_symbol} {self.steps[idx]}</div>'
        elif status == 'completed':
            self.step_widgets[idx].value = f'<div class="step completed">✓ {self.steps[idx]}</div>'
        else:
            self.step_widgets[idx].value = f'<div class="step">⭕ {self.steps[idx]}</div>'
        self.update_step_display()
    
    def update_step_display(self):
        """Refresh step indicator"""
        self.step_indicator.value = '<div class="step-indicator">' + \
                                    ''.join([w.value for w in self.step_widgets]) + \
                                    '</div>'
    
    def display_assets(self, measured_assets):
        """Displays assets - THREAD-SAFE using HTML widget"""
        import base64
        
        print("\n🔍 DISPLAY_ASSETS called")
        print(f"🔍 Thread: {threading.current_thread().name}")
        
        if not isinstance(measured_assets, dict):
            self.asset_display.value = "<div style='color:red; padding:20px;'>❌ measured_assets is not a dict</div>"
            return

        assets = measured_assets.get("assets_mapping", {})
        clip_map = measured_assets.get("clip_thumbnails", {}) or {}
        
        print(f"🔍 Found {len(assets)} assets")

        if not assets:
            self.asset_display.value = "<div style='color:orange; padding:20px;'>⚠️ No assets found</div>"
            return

        # Build complete HTML
        all_html = [f"""
        <div style='background:#e8f5e9; padding:15px; border-radius:8px; margin-bottom:15px; border:2px solid #4caf50;'>
            <strong style='font-size:18px;'>✅ Found {len(assets)} assets</strong>
        </div>
        """]

        for idx, (asset_name, info) in enumerate(assets.items()):
            print(f"🔍 Processing {idx+1}: {asset_name}")
            
            display_name = info.get("asset_name", asset_name)
            thumbnail_path = info.get("thumbnail", "")
            
            if thumbnail_path:
                thumbnail_path = os.path.normpath(os.path.abspath(thumbnail_path))
            
            # Base64 encode
            img_html = '<div style="color:#9ca3af; text-align:center; padding:40px;">No Image</div>'
            if thumbnail_path and os.path.exists(thumbnail_path):
                try:
                    with open(thumbnail_path, "rb") as f:
                        encoded = base64.b64encode(f.read()).decode("ascii")
                        mime = "image/png" if thumbnail_path.lower().endswith(".png") else "image/jpeg"
                        img_html = f'<img src="data:{mime};base64,{encoded}" style="width:120px;height:120px;object-fit:contain;">'
                        print(f"🔍   ✅ Encoded image")
                except Exception as e:
                    print(f"🔍   ❌ Error: {e}")

            clip_score = 0.0
            if clip_map and asset_name in clip_map:
                clip_score = float(clip_map[asset_name].get("clip_score", 0.0))

            dim = info.get("measured_dimensions", {})
            dx, dy, dz = dim.get("x", 0.0), dim.get("y", 0.0), dim.get("z", 0.0)

            all_html.append(f"""
            <div style="width:100%; padding:14px; margin:10px 0; border-radius:12px; background:white;
                        border:2px solid #4caf50; display:flex; gap:15px; align-items:center; 
                        box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                <div style="width:120px; height:120px; background:#f3f4f6; border:2px solid #ddd;
                            display:flex; justify-content:center; align-items:center; 
                            overflow:hidden; border-radius:8px;">
                    {img_html}
                </div>
                <div style="flex:1;">
                    <div style="font-size:18px; font-weight:700; color:#1f2937; margin-bottom:8px;">
                        {display_name}
                    </div>
                    <div style="margin-top:6px;">
                        <span style="background:#d1fae5; padding:6px 14px; border-radius:20px; 
                                    font-size:13px; font-weight:600; color:#065f46;">
                            CLIP Score: {clip_score:.3f}
                        </span>
                    </div>
                    <div style="margin-top:10px; font-size:13px; color:#6b7280; font-weight:500;">
                        📏 Dimensions: {dx:.1f} × {dy:.1f} × {dz:.1f} m
                    </div>
                </div>
            </div>
            """)

        combined_html = "\n".join(all_html)
        print(f"🔍 Setting HTML ({len(combined_html)} chars)")
        
        # THREAD-SAFE: Set widget value directly
        self.asset_display.value = combined_html
        print("🔍 ✅ Done!\n")
    
    def display_render(self, scene_folder):
        """Display rendered scene preview - THREAD-SAFE"""
        import base64
        
        print("\n🎬 DISPLAY_RENDER called")
        print(f"🎬 Scene folder: {scene_folder}")
        
        possible_renders = [
            os.path.join(scene_folder, "overhead_view.png"),
            os.path.join(scene_folder, "ground_view.png")
        ]
        
        render_html = []
        shown = False
        
        for render_path in possible_renders:
            print(f"🎬 Checking: {render_path}")
            
            # Normalize path
            if os.path.exists(render_path):
                print(f"🎬 ✅ Found: {render_path}")
                try:
                    # Read and base64 encode the image
                    with open(render_path, "rb") as f:
                        img_data = f.read()
                        encoded = base64.b64encode(img_data).decode("ascii")
                        mime = "image/png" if render_path.lower().endswith(".png") else "image/jpeg"
                        img_src = f"data:{mime};base64,{encoded}"
                        
                        render_name = os.path.basename(render_path).replace("_", " ").replace(".png", "").title()
                        
                        render_html.append(f"""
                        <div style="margin:20px 0;">
                            <h4 style="color:#1f2937; margin-bottom:10px;">{render_name}</h4>
                            <img src="{img_src}" style="max-width:100%; border-radius:12px; 
                                box-shadow: 0 10px 30px rgba(0,0,0,0.2);">
                        </div>
                        """)
                        shown = True
                        print(f"🎬 ✅ Encoded {len(img_data)} bytes")
                except Exception as e:
                    print(f"🎬 ❌ Error encoding {render_path}: {e}")
            else:
                print(f"🎬 ❌ Not found: {render_path}")
        
        if not shown:
            # Try searching in subdirectories
            print("🎬 Searching subdirectories...")
            for root, dirs, files in os.walk(scene_folder):
                for file in files:
                    if file in ["overhead_view.png", "ground_view.png"]:
                        full_path = os.path.join(root, file)
                        print(f"🎬 Found in subdirectory: {full_path}")
                        try:
                            with open(full_path, "rb") as f:
                                img_data = f.read()
                                encoded = base64.b64encode(img_data).decode("ascii")
                                mime = "image/png"
                                img_src = f"data:{mime};base64,{encoded}"
                                
                                render_name = file.replace("_", " ").replace(".png", "").title()
                                render_html.append(f"""
                                <div style="margin:20px 0;">
                                    <h4 style="color:#1f2937; margin-bottom:10px;">{render_name}</h4>
                                    <img src="{img_src}" style="max-width:100%; border-radius:12px; 
                                        box-shadow: 0 10px 30px rgba(0,0,0,0.2);">
                                </div>
                                """)
                                shown = True
                                print(f"🎬 ✅ Encoded {len(img_data)} bytes")
                        except Exception as e:
                            print(f"🎬 ❌ Error: {e}")
        
        if shown:
            combined_html = "\n".join(render_html)
            self.render_preview.value = combined_html
            print("🎬 ✅ Renders displayed!")
        else:
            self.render_preview.value = '<div style="color:#6b7280; padding:20px;">No render images found yet.</div>'
            print("🎬 ⚠️ No renders found")
    
    def start_generation(self, btn):
        """Start scene generation"""
        prompt = self.prompt_input.value.strip()
        if not prompt:
            self.add_status('⚠️ Please enter a scene description', 'warning')
            return
        
        self.generation_running = True
        self.generate_btn.disabled = True
        self.generate_btn.description = '⏳ Generating...'
        
        # Clear previous outputs
        with self.status_output:
            clear_output()
        
        self.code_display.value = ''
        self.copy_btn.layout.display = 'none'
        self.progress_bar.value = 0
        
        # Reset steps
        for i, step in enumerate(self.steps):
            self.step_widgets[i].value = f'<div class="step">{step}</div>'
        self.update_step_display()
        
        # Run in background thread
        thread = threading.Thread(target=self.run_pipeline, args=(prompt,))
        thread.daemon = True
        thread.start()
    
    def run_pipeline(self, original_prompt):
        """Execute the complete generation pipeline"""
        try:
            pipeline_start_time = time.time()
            generation_id = None  # ✅ ADD THIS
            correction_count = 0  # ✅ ADD THIS
            
            self.add_status(f'🚀 Starting generation for: "{original_prompt}"', 'info')
            # STEP 1: Meta-Learning
            self.update_step(0, 'active')
            self.add_status('🧠 Analyzing historical data for similar scenes...', 'info')
            self.progress_bar.value = 5
            
            if ENABLE_META_LEARNING:
                prompt = enhance_prompt_with_meta_learning(original_prompt)
                if prompt != original_prompt:
                    self.add_status(f'✅ Prompt enhanced with learned insights', 'success')
                    self.add_status(f'💡 Enhanced: {prompt}', 'info')
                else:
                    self.add_status('ℹ️ No similar scenes found - using original prompt', 'info')
                    prompt = original_prompt
            else:
                prompt = original_prompt
            
            self.update_step(0, 'completed')
            self.progress_bar.value = 10
            
            # Create project directory
            import re
            safe_folder = re.sub(r'[^\w\s-]', '', original_prompt).strip()
            safe_folder = re.sub(r'[-\s]+', '_', safe_folder).lower()[:100]
            project_dir = safe_folder
            assets_dir = os.path.join(project_dir, "assets")
            scene_dir = os.path.join(project_dir, "scene")
            os.makedirs(assets_dir, exist_ok=True)
            os.makedirs(scene_dir, exist_ok=True)
            
            # STEP 2: Asset Identification
            self.update_step(1, 'active')
            self.add_status('🔍 Identifying key 3D assets needed...', 'info')
            self.progress_bar.value = 15
            
            asset_output_raw = get_agent_response(asset_identifier_agent, f"Identify 3-5 key 3D assets for a '{prompt}' scene.")
            asset_list = parse_json_safely(extract_content_from_response(asset_output_raw))
            
            if isinstance(asset_list, list):
                asset_names = [a.get('asset_name', 'Unknown') for a in asset_list]
                self.add_status(f'✅ Identified {len(asset_list)} assets: {", ".join(asset_names)}', 'success')
            else:
                self.add_status('❌ Failed to identify assets', 'error')
                return
            
            self.update_step(1, 'completed')
            self.progress_bar.value = 20
            
            # STEP 2.5: Spatial Analysis
            self.add_status('🎯 Analyzing spatial relationships...', 'info')
            spatial_input = f"Analyze spatial relationships in: '{prompt}'. Assets: {safe_json_dumps(asset_list)}"
            spatial_output_raw = get_agent_response(spatial_analyzer_agent, spatial_input)
            spatial_constraints = parse_json_safely(extract_content_from_response(spatial_output_raw))
            
            # STEP 3: Download & Verify
            self.update_step(2, 'active')
            self.add_status('📦 Searching and downloading assets from Sketchfab...', 'info')
            self.progress_bar.value = 30
            
            asset_fetcher_result = process_asset_list_sketchfab(asset_list, assets_dir)
            assets_found = asset_fetcher_result.get("total_assets_found", 0)
            
            if assets_found == 0:
                self.add_status('❌ No assets could be downloaded', 'error')
                return
            
            self.add_status(f'✅ Downloaded {assets_found} assets successfully', 'success')
            self.update_step(2, 'completed')
            self.progress_bar.value = 40
            
            # STEP 4: Measure Assets
            self.update_step(3, 'active')
            self.add_status('📏 Measuring asset dimensions in Blender...', 'info')
            self.progress_bar.value = 45
            
            assets_to_measure = asset_fetcher_result.get("assets_mapping", {})
            measured_assets = measure_assets_in_blender(assets_to_measure, project_dir, BLENDER_EXECUTABLE_PATH)
            # quick debug output to notebook + dashboard status
            try:
                self.add_status(f'🛠️ DEBUG: measured_assets type={type(measured_assets)}', 'info')
                print("DEBUG measured_assets (repr):", repr(measured_assets)[:1000])
                if measured_assets is None:
                    self.add_status('🛠️ DEBUG: measured_assets is None', 'warning')
                elif isinstance(measured_assets, dict) and not measured_assets:
                    self.add_status('🛠️ DEBUG: measured_assets is empty dict', 'warning')
                else:
                    # show top-level keys if dict
                    if isinstance(measured_assets, dict):
                        keys = list(measured_assets.keys())
                        self.add_status(f'🛠️ DEBUG: measured_assets keys={keys[:10]}', 'info')
            except Exception as _err:
                print("DEBUG: failed to emit measured_assets debug:", _err)
            
            if not measured_assets:
                self.add_status('❌ Asset measurement failed', 'error')
                return
            
            self.add_status('✅ Assets measured and validated with CLIP', 'success')
            self.update_step(3, 'completed')
            self.progress_bar.value = 55
            
            # Display assets
            self.display_assets(measured_assets)
            
            if ENABLE_DATABASE:
                try:
                    generation_id = store_generation_result(
                        prompt=original_prompt,
                        success=False,
                        project_path=os.path.abspath(project_dir),
                        generation_time_seconds=0,
                        correction_attempts=0,
                        final_attempt_number=0,
                        assets_requested=len(asset_list),
                        assets_found=assets_found,
                        assets_used=len(measured_assets.get('assets_mapping', {}))
                    )
                    print(f"📊 Created database record (ID: {generation_id})")
                except Exception as e:
                    print(f"⚠️ Database storage failed: {e}")
                    generation_id = None

            # STEP 5: Planning
            self.update_step(4, 'active')
            self.add_status('🎯 Creating detailed placement plan...', 'info')
            self.progress_bar.value = 60
            
            # High-level planning
            high_level_input = f"Create a high-level plan for '{prompt}' with assets: {safe_json_dumps(asset_list)}"
            high_level_raw = get_agent_response(high_level_planner_agent, high_level_input)
            high_level_plan = parse_json_safely(extract_content_from_response(high_level_raw))
            
            # Detailed planning
            detailed_input = (f"Create detailed placement plan for '{prompt}'.\n"
                            f"High-level: {safe_json_dumps(high_level_plan)}\n"
                            f"Spatial constraints: {safe_json_dumps(spatial_constraints)}\n"
                            f"Assets with dimensions: {safe_json_dumps(measured_assets)}")
            detailed_raw = get_agent_response(detailed_planner_agent, detailed_input)
            detailed_plan = parse_json_safely(extract_content_from_response(detailed_raw))
            
            self.add_status('✅ Spatial layout optimized', 'success')
            self.update_step(4, 'completed')
            self.progress_bar.value = 70
            
            # STEP 6: Code Generation
            self.update_step(5, 'active')
            self.add_status('💻 Generating Blender Python script...', 'info')
            self.progress_bar.value = 75
            
            codegen_input = {
                "detailed_plan": detailed_plan,
                "spatial_constraints": spatial_constraints,
                "fetched_assets": measured_assets.get("assets_mapping", {}),
                "original_prompt": prompt
            }
            
            codegen_raw = get_agent_response(code_generator_agent, json.dumps(codegen_input))
            codegen_result = parse_json_safely(extract_content_from_response(codegen_raw))
            
            if "script" not in codegen_result:
                self.add_status('❌ Code generation failed', 'error')
                return

            # keep generated script in memory but DO NOT display yet
            script_content = codegen_result["script"]
            # don't set self.code_display or show copy button here
            # self.code_display.value = script_content
            # self.copy_btn.layout.display = 'inline-block'
            
            self.add_status('✅ Blender script generated successfully', 'success')
            self.update_step(5, 'completed')
            self.progress_bar.value = 80
            
            # STEP 7: Correction Loop
            self.update_step(6, 'active')
            self.add_status('🔧 Testing and correcting script...', 'info')
            
            max_attempts = 10
            correction_count = 0
            error_output = None

            for attempt in range(max_attempts):
                self.add_status(f'🔄 Attempt {attempt + 1}/{max_attempts}...', 'info')
                error_output = run_blender_script(script_content, project_dir, BLENDER_EXECUTABLE_PATH)

                if ENABLE_DATABASE and generation_id:
                    try:
                        store_correction_attempt(
                            generation_id=generation_id,
                            attempt_number=attempt + 1,
                            agent_used='code_corrector_agent' if attempt > 0 else 'code_generator_agent',
                            error_occurred=(error_output is not None),
                            error_traceback=error_output,
                            execution_time_seconds=None
                        )
                    except Exception as e:
                        print(f"⚠️ Failed to log attempt: {e}")

                if error_output is None:
                    # success, show final working script
                    self.add_status('✅ Script executed successfully!', 'success')
                    # display the final runnable script and enable copy button
                    self.code_display.value = script_content
                    self.copy_btn.layout.display = 'inline-block'
                    break

                correction_count += 1
                self.add_status(f'⚠️ Error detected, attempting correction #{correction_count}', 'warning')

                # Call corrector
                correction_input = f"Fix this error:\n{error_output}\n\nScript:\n{script_content}"
                correction_raw = get_agent_response(code_corrector_agent, correction_input)
                correction_result = parse_json_safely(extract_content_from_response(correction_raw))

                if "script" in correction_result:
                    script_content = correction_result["script"]
                    # DO NOT display intermediate corrected scripts — wait until a successful run
                    time.sleep(65)  # Rate limit
                else:
                    break

            if error_output is not None:
                # failed after attempts
                self.add_status(f'❌ Failed after {max_attempts} correction attempts', 'error')
                # show failure marker in code area and hide copy button
                self.code_display.value = '❌ Failed to produce a runnable Blender script after automated corrections.'
                self.copy_btn.layout.display = 'none'
                return

            self.update_step(6, 'completed')
            self.progress_bar.value = 90 
            
            # STEP 8: Validation
            self.update_step(7, 'active')
            self.add_status('🎯 Running CLIP validation on rendered scene...', 'info')
            self.progress_bar.value = 95
            
            clip_score = validate_scene_with_clip(original_prompt, scene_dir, clip_threshold=0.25)
            
            if clip_score >= 0.25:
                self.add_status(f'✅ Validation PASSED! CLIP Score: {clip_score:.3f}', 'success')
            else:
                self.add_status(f'⚠️ Validation score below threshold: {clip_score:.3f}', 'warning')

            if ENABLE_DATABASE and generation_id:
                try:
                    blender_metrics = extract_scene_metrics_from_blender(scene_dir, BLENDER_EXECUTABLE_PATH)
                    
                    # Update generation record with final data
                    import psycopg2
                    conn = psycopg2.connect(**DB_CONFIG)
                    cur = conn.cursor()
                    
                    generation_time = time.time() - pipeline_start_time
                    
                    cur.execute("""
                        UPDATE generations SET
                            success = TRUE,
                            correction_attempts = %s,
                            generation_time_seconds = %s,
                            clip_scene_score = %s,
                            validation_passed = %s,
                            terrain_relief_m = %s,
                            terrain_rms_disp = %s,
                            mean_nnd_xy = %s,
                            global_density_per_m2 = %s,
                            out_of_bounds_count = %s,
                            projected_coverage_pct = %s,
                            total_vertices = %s,
                            total_triangles = %s
                        WHERE id = %s
                    """, (
                        correction_count,
                        generation_time,
                        clip_score,
                        clip_score >= 0.25,
                        blender_metrics.get('terrain_relief_m') if blender_metrics else None,
                        blender_metrics.get('terrain_rms_disp') if blender_metrics else None,
                        blender_metrics.get('mean_nnd_xy') if blender_metrics else None,
                        blender_metrics.get('global_density_per_m2') if blender_metrics else None,
                        blender_metrics.get('out_of_bounds_count') if blender_metrics else None,
                        blender_metrics.get('projected_coverage_pct') if blender_metrics else None,
                        blender_metrics.get('total_vertices') if blender_metrics else None,
                        blender_metrics.get('total_triangles') if blender_metrics else None,
                        generation_id
                    ))
                    
                    conn.commit()
                    cur.close()
                    conn.close()
                    
                    # Store asset details
                    store_asset_details(generation_id, measured_assets, detailed_plan)
                    
                    # Update asset performance
                    for asset_name in measured_assets.get('assets_mapping', {}).keys():
                        clip_data = measured_assets.get('clip_thumbnails', {}).get(asset_name, {})
                        update_asset_performance(
                            asset_name=asset_name,
                            success=True,
                            clip_score=clip_data.get('clip_score')
                        )
                    
                    self.add_status(f'💾 Data saved to database (ID: {generation_id})', 'success')
                    print(f"✅ Database updated successfully (Generation ID: {generation_id})")
                    
                except Exception as e:
                    print(f"⚠️ Database update failed: {e}")
                    import traceback
                    traceback.print_exc()


            self.update_step(7, 'completed')
            self.progress_bar.value = 100
            
            # Display renders
            self.display_render(scene_dir)
            
            # Final success message
            generation_time = time.time() - pipeline_start_time
            self.add_status(f'🎉 Scene generation complete in {generation_time:.1f}s!', 'success')
            self.add_status(f'📁 Project saved to: {os.path.abspath(project_dir)}', 'info')
            
        except Exception as e:
            import traceback
            self.add_status(f'❌ Pipeline error: {str(e)}', 'error')
            print(traceback.format_exc())
            
            # ✅ ADD THIS - Log failure to database
            if ENABLE_DATABASE and generation_id:
                try:
                    generation_time = time.time() - pipeline_start_time
                    import psycopg2
                    conn = psycopg2.connect(**DB_CONFIG)
                    cur = conn.cursor()
                    cur.execute("""
                        UPDATE generations SET
                            success = FALSE,
                            correction_attempts = %s,
                            generation_time_seconds = %s,
                            final_error_log = %s,
                            error_type = 'pipeline_exception'
                        WHERE id = %s
                    """, (correction_count, generation_time, str(e), generation_id))
                    conn.commit()
                    cur.close()
                    conn.close()
                except Exception as db_error:
                    print(f"⚠️ Failed to log error to database: {db_error}")
        finally:
            self.generation_running = False
            self.generate_btn.disabled = False
            self.generate_btn.description = '✨ Generate Scene'
    
    def copy_code(self, btn):
        """Display code in a selectable textarea for manual copying - Voilà compatible"""
        
        # Create a modal-style overlay with selectable text
        code = self.code_display.value
        
        # Create a new textarea that's NOT disabled, so users can select and copy
        copy_area = widgets.Textarea(
            value=code,
            layout=widgets.Layout(width='90%', height='500px'),
            disabled=False  # Important: must be enabled for selection
        )
        
        instruction_html = widgets.HTML("""
        <div style='background:#e8f5e9; border:2px solid #4caf50; border-radius:8px; 
                    padding:20px; margin:20px; text-align:center;'>
            <h3 style='margin-top:0; color:#065f46;'>📋 Copy Your Code</h3>
            <p style='font-size:16px; margin:10px 0;'>
                <strong>Step 1:</strong> Click inside the code area below<br>
                <strong>Step 2:</strong> Press <kbd style='background:#fff; padding:4px 8px; 
                        border:1px solid #ddd; border-radius:4px;'>Ctrl+A</kbd> 
                (or <kbd style='background:#fff; padding:4px 8px; border:1px solid #ddd; 
                        border-radius:4px;'>Cmd+A</kbd> on Mac) to select all<br>
                <strong>Step 3:</strong> Press <kbd style='background:#fff; padding:4px 8px; 
                        border:1px solid #ddd; border-radius:4px;'>Ctrl+C</kbd> 
                (or <kbd style='background:#fff; padding:4px 8px; border:1px solid #ddd; 
                        border-radius:4px;'>Cmd+C</kbd> on Mac) to copy
            </p>
        </div>
        """)
        
        close_btn = widgets.Button(
            description='✓ Done',
            button_style='success',
            layout=widgets.Layout(width='200px', height='50px')
        )
        
        modal_container = widgets.VBox([
            instruction_html,
            copy_area,
            widgets.HBox([close_btn], layout=widgets.Layout(justify_content='center', margin='20px'))
        ], layout=widgets.Layout(
            border='3px solid #4caf50',
            border_radius='15px',
            padding='20px',
            background_color='white'
        ))
        
        # Create overlay output
        overlay_output = widgets.Output()
        
        def close_modal(b):
            overlay_output.clear_output()
            # Remove from main container
            items = list(self.main_container.children)
            if overlay_output in items:
                items.remove(overlay_output)
                self.main_container.children = tuple(items)
            btn.description = '📋 Copy Code'
        
        close_btn.on_click(close_modal)
        
        # Display modal
        with overlay_output:
            display(modal_container)
        
        # Add to main container
        items = list(self.main_container.children)
        # Insert before code display
        for i, child in enumerate(items):
            if child == self.code_display:
                items.insert(i, overlay_output)
                break
        self.main_container.children = tuple(items)
        
        btn.description = '👆 See above'
    
    def display(self):
        root = widgets.VBox([
            widgets.HTML(CSS_STYLES),
            self.header,
            self.main_container
        ], layout=widgets.Layout(
            max_width='1400px',
            margin='0 auto',
            background_color='white',
            border_radius='20px',
            padding='40px'
        ))
        display(root)


# Launch dashboard
if __name__ == "__main__":
    print("🚀 Launching AI 3D Scene Generator Dashboard...")
    dashboard = SceneGeneratorDashboard()
    dashboard.display()